In [9]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import statsmodels.api as sm
import numpy as np
import geopandas as gpd
from ipfn import ipfn
from matplotlib import pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'
from sttn.data.lehd import OriginDestinationEmploymentDataProvider
provider = OriginDestinationEmploymentDataProvider()
import os
import math
from sttn.network import SpatioTemporalNetwork
from sttn.utils import add_distance

%matplotlib inline

In [4]:
cities = [
    ('New York City', 'ny', ['New York County, NY', 'Queens County, NY','Kings County, NY','Bronx County, NY','Richmond County, NY']),
    ('Los Angeles', 'ca', ['Los Angeles County, CA']),
    ('Chicago', 'il', ['Cook County, IL']),
    ('Houston', 'tx', ['Harris County, TX']),
    ('Boston', 'ma', ['Suffolk County, MA', 'Middlesex County, MA']),
    ('Phoenix', 'az', ['Maricopa County, AZ']),
    ('Philadelphia', 'pa', ['Philadelphia County, PA']),
    ('San Antonio', 'tx', ['Bexar County, TX']),
    ('San Diego', 'ca', ['San Diego County, CA']),
    ('Dallas', 'tx', ['Dallas County, TX']),
    ('San Jose', 'ca', ['Santa Clara County, CA']),
    ('Austin', 'tx', ['Travis County, TX']),
]

In [10]:
# get data for above cities - census tract level

allCity_dfs = []
job_column = 'S000'
comp_aggs={job_column: 'sum'}
for city, state, conties in cities:
    state_network = provider.get_data(state=state, year=2018)
    city_network = state_network.filter_nodes(state_network.nodes.county.isin(conties))
    with_distance = add_distance(city_network).edges
    
    city_jobs = city_network.agg_adjacent_edges(aggs=comp_aggs, outgoing=False).rename(columns={job_column: 'jobs'}).reset_index()
    city_pop = city_network.agg_adjacent_edges(aggs=comp_aggs, outgoing=True).rename(columns={job_column: 'residence'}).reset_index()
    
    city_dist = with_distance.merge(city_jobs, on='destination')
    city_cum = city_dist.merge(city_pop, on='origin')
    
    allCity_dfs.append(city_cum)

In [13]:
!mkdir cities
for i,df in enumerate(allCity_dfs):
    city = cities[i][0]
    df.to_csv('cities/'+city+'.csv')

mkdir: cannot create directory ‘cities’: File exists


### preprocessing

In [35]:
def preprocessing(data, separate_income=False):
    y_target = ['S000']  # target = total commute if no income segregation
    origin = df.groupby(['origin']).agg({'S000':sum}).reset_index()
    origin.columns = ['origin','S000residence']
    destination = df.groupby(['destination']).agg({'S000':sum}).reset_index()
    destination.columns = ['destination','S000jobs']
    data = data.merge(origin,on=['origin'])
    data = data.merge(destination,on=['destination'])
    if separate_income == True:
        
        y_target = ['SE01', 'SE02', 'SE03'] 
        origin = df.groupby(['origin']).agg({'SE01':sum,'SE02':sum,'SE03':sum}).reset_index()
        origin.columns = ['origin','SE01residence','SE02residence','SE03residence']
        destination = df.groupby(['destination']).agg({'SE01':sum,'SE02':sum,'SE03':sum}).reset_index()
        destination.columns = ['destination','SE01jobs','SE02jobs','SE03jobs']
        data = data.merge(origin,on=['origin'])
        data = data.merge(destination,on=['destination'])
    return data

In [20]:
!mkdir citiesProcessed/
cities = os.listdir('cities/')
for city in cities:
    if '.csv' in city:
        df = pd.read_csv('cities/'+city)
        df = preprocessing(df,separate_income=False)
        df = df[['origin','destination','distance','S000residence','S000jobs','S000']]
        df.to_csv('citiesProcessed/'+city,index=False)


mkdir: cannot create directory ‘citiesProcessed/’: File exists


In [23]:
!mkdir citiesProcessedIncome/
cities = os.listdir('cities/')
for city in cities:
    if '.csv' in city:
        df = pd.read_csv('cities/'+city)
        df = preprocessing(df,separate_income=True)
        df = df[['origin','SE01residence','SE02residence','SE03residence','destination','SE01jobs','SE02jobs','SE03jobs','distance','S000','SE01', 'SE02', 'SE03']]
        df.to_csv('citiesProcessedIncome/'+city,index=False)

mkdir: cannot create directory ‘citiesProcessedIncome/’: File exists


### MSE, doubly constrained, fit u,v together in iterations, from Mingyi

In [56]:
def getbins(df, nbins=20):     
    df['bin'] = pd.qcut(df['distance'], q=20)
    df.sort_values(by='bin', inplace=True)
    df.rename(columns={'jobs':'S000jobs', 'residence':'S000residence'}, inplace=True)
    
    return df
def balancing(test,target,iterationNum,iteration = 20):
#     print(target,'iteration', iterationNum)
    if target+'B' not in test.columns:
        test[target+'B'] = 1
    test[target+'BDF'] = test[target+'jobs']*test[target+'f(d)']*test[target+'B']
    if target+'A' in test.columns:
        del test[target+'A']
    del test[target+'B']
    test = test.groupby(['origin']).agg({target+'BDF':sum}).\
    rename(columns={target+'BDF':target+'A'}).reset_index().\
    merge(test,on=['origin'],how='right')
    test[target+'A'] = 1/test[target+'A']
    test[target+'AOF'] = test[target+'residence']*test[target+'f(d)']*test[target+'A']
    test = test.groupby(['destination']).agg({target+'AOF':sum}).\
    rename(columns={target+'AOF':target+'B'}).reset_index().\
    merge(test,on=['destination'],how='right')
    test[target+'B'] = 1/test[target+'B']
    test[target+'flowPred'] = test[target+'residence']*test[target+'jobs']*test[target+'f(d)']*\
                        test[target+'A']*test[target+'B']
    
    resultO = test[['origin',target+'residence']].drop_duplicates().\
    merge(test.groupby(['origin'])[[target+'flowPred']].sum().reset_index(),on=['origin'],how='left')
    resultO['percentage'] = np.abs(resultO[target+'residence'] - resultO[target+'flowPred'])/resultO[target+'residence']
    resultO = resultO['percentage'].mean()

    resultD = test[['destination',target+'jobs']].drop_duplicates().\
    merge(test.groupby(['destination'])[[target+'flowPred']].sum().reset_index(),on=['destination'],how='left')
    resultD['percentage'] = np.abs(resultD[target+'jobs'] - resultD[target+'flowPred'])/resultD[target+'jobs']
    resultD = resultD['percentage'].mean()
#     print(resultO,resultD)
    if resultO < 0.05 and resultD < 0.05:
        return test
    else:
        if iterationNum < iteration:
            return balancing(test,target,iterationNum = iterationNum+1,iteration = 20)
        else:
            return test
        
def doubly_constrained_model_AB(data, separate_income=False):
    
    y_target = ['S000']
    if separate_income == True:
        y_target = ['SE01', 'SE02', 'SE03'] 
    targetOutput = []
    for target in y_target:
        binoutput = pd.DataFrame()
        # estimate F for each bin
        for b in data['bin'].unique():
            subData = data[data['bin'] == b]
            X = subData[target+'residence'] * subData[target+'jobs']
            
            y = subData[target]

            model = sm.OLS(y,X).fit()
            
            subData[target+'f(d)'] = model.params[0]       
            binoutput = pd.concat([binoutput,subData])
        binoutput = balancing(binoutput,target,iterationNum=1,iteration = 20)
        
        binoutput = binoutput[['origin','destination',target,target+'A',target+'B',target+'f(d)','bin',target+'flowPred']]
        targetOutput.append(binoutput)
    if separate_income == True:
        targetOutput = targetOutput[0].merge(targetOutput[1],on=['origin','destination'],how='outer').\
                        merge(targetOutput[2],on=['origin','destination'],how='outer')
    else:
        targetOutput = targetOutput[0]
    targetOutput = targetOutput.merge(data[['origin','destination','S000']])
    return targetOutput

In [58]:
!mkdir constrainCTdistbinsAB/
!mkdir constrainCTdistbinsABIncome/
separate_income = True
if separate_income == True:
    citiesDir = 'citiesProcessedIncome/'
    outputDir = 'constrainCTdistbinsABIncome/'
else:
    citiesDir = 'citiesProcessed/'
    outputDir = 'constrainCTdistbinsAB/'
for city in cities:
    if '.csv' in city:
        df = pd.read_csv(citiesDir+city)
        dataUV = doubly_constrained_model_AB(getbins(df),separate_income=separate_income)
        dataUV.to_csv(outputDir+city,index=False)


mkdir: cannot create directory ‘constrainCTdistbinsAB/’: File exists
mkdir: cannot create directory ‘constrainCTdistbinsABIncome/’: File exists


In [57]:
separate_income = False
if separate_income == True:
    citiesDir = 'citiesProcessedIncome/'
    outputDir = 'constrainCTdistbinsABIncome/'
else:
    citiesDir = 'citiesProcessed/'
    outputDir = 'constrainCTdistbinsAB/'
for city in cities:
    if '.csv' in city:
        df = pd.read_csv(citiesDir+city)
        dataUV = doubly_constrained_model_AB(getbins(df),separate_income=separate_income)
        dataUV.to_csv(outputDir+city,index=False)

### unconstrain model, power law, from Mingyi

In [63]:
import scipy.optimize as optimize
def power_law(x,k,a):
    return k*((x[:,0]**a)*x[:,1]*x[:,2])
def unconstrained_powerlaw(data, separate_income=False):
    y_target = ['S000']
    if separate_income == True:
        y_target = ['SE01', 'SE02', 'SE03'] 
    dataF = []
    for target in y_target:
        X = data[['distance',target+'jobs',target+'residence']].values
        y = data[target].values
        pars, cov = optimize.curve_fit(f=power_law, xdata=X, ydata=y, bounds=(-np.inf, np.inf))
#         print(pars)
        data[target+'k'] = pars[0]
        data[target+'a'] = pars[1]
        data[target+'pred'] = data[target+'k']*(data['distance']**data[target+'a'])*data[target+'jobs']*data[target+'residence']
    return data

        

In [64]:
!mkdir unconstrainCTPowerlaw/


separate_income = False
if separate_income == True:
    citiesDir = 'citiesProcessedIncome/'
    outputDir = 'unconstrainCTPowerlawIncome/'
else:
    citiesDir = 'citiesProcessed/'
    outputDir = 'unconstrainCTPowerlaw/'
for city in cities:
    if '.csv' in city:
        df = pd.read_csv(citiesDir+city)
        dataUV = unconstrained_powerlaw(df,separate_income=separate_income)
        dataUV.to_csv(outputDir+city,index=False)


mkdir: cannot create directory ‘unconstrainCTPowerlaw/’: File exists


In [67]:
!mkdir unconstrainCTPowerlawIncome/
separate_income = True
if separate_income == True:
    citiesDir = 'citiesProcessedIncome/'
    outputDir = 'unconstrainCTPowerlawIncome/'
else:
    citiesDir = 'citiesProcessed/'
    outputDir = 'unconstrainCTPowerlaw/'
for city in cities:
    if '.csv' in city:
        df = pd.read_csv(citiesDir+city)
        dataUV = unconstrained_powerlaw(df,separate_income=separate_income)
        dataUV.to_csv(outputDir+city,index=False)

### unconstrain model, full power law, from Mingyi

In [68]:

def power_law(x,k,a,b,c):
    return k*(x[:,0]**a)*(x[:,1]**b)*(x[:,2]**c)
def unconstrained_fullpowerlaw(data, separate_income=False):
    y_target = ['S000']
    if separate_income == True:
        y_target = ['SE01', 'SE02', 'SE03'] 
    dataF = []
    for target in y_target:
        X = data[['distance',target+'jobs',target+'residence']].values
        y = data[target].values
        pars, cov = optimize.curve_fit(f=power_law, xdata=X, ydata=y, bounds=(-np.inf, np.inf))
#         print(pars)
        data[target+'k'] = pars[0]
        data[target+'a'] = pars[1]
        data[target+'b'] = pars[2]
        data[target+'c'] = pars[3]
        data[target+'pred'] = data[target+'k']*(data['distance']**data[target+'a'])*\
                        (data[target+'jobs']**data[target+'b'])*(data[target+'residence']**data[target+'c'])
    return data

        

In [69]:
!mkdir unconstrainCTFullPowerlaw/
!mkdir unconstrainCTFullPowerlawIncome/


separate_income = False
if separate_income == True:
    citiesDir = 'citiesProcessedIncome/'
    outputDir = 'unconstrainCTFullPowerlawIncome/'
else:
    citiesDir = 'citiesProcessed/'
    outputDir = 'unconstrainCTFullPowerlaw/'
for city in cities:
    if '.csv' in city:
        df = pd.read_csv(citiesDir+city)
        dataUV = unconstrained_fullpowerlaw(df,separate_income=separate_income)
        dataUV.to_csv(outputDir+city,index=False)


In [70]:

separate_income = True
if separate_income == True:
    citiesDir = 'citiesProcessedIncome/'
    outputDir = 'unconstrainCTFullPowerlawIncome/'
else:
    citiesDir = 'citiesProcessed/'
    outputDir = 'unconstrainCTFullPowerlaw/'
for city in cities:
    if '.csv' in city:
        df = pd.read_csv(citiesDir+city)
        dataUV = unconstrained_fullpowerlaw(df,separate_income=separate_income)
        dataUV.to_csv(outputDir+city,index=False)


### unconstrain model, exp, from Mingyi

In [72]:
def exp(x, a,b):
    return a*(np.e**(b*x))
def unconstrained_exp(data, separate_income=False):
    y_target = ['S000']
    if separate_income == True:
        y_target = ['SE01', 'SE02', 'SE03'] 
    dataF = []
    for target in y_target:
        X = data.distance.values
        y = data[target]/(data[target+'jobs']*data[target+'residence'])
        pars, cov = optimize.curve_fit(f=exp, xdata=X, ydata=y, bounds=(-np.inf, np.inf))
#         print(pars)
        data[target+'a'] = pars[0]
        data[target+'b'] = pars[1]
        data[target+'pred'] = data[target+'a']*(np.e**(data['distance']*data[target+'b']))*data[target+'jobs']*data[target+'residence']
    return data

        

In [73]:
!mkdir unconstrainCTFullExp/
!mkdir unconstrainCTFullExpIncome/


separate_income = False
if separate_income == True:
    citiesDir = 'citiesProcessedIncome/'
    outputDir = 'unconstrainCTFullExpIncome/'
else:
    citiesDir = 'citiesProcessed/'
    outputDir = 'unconstrainCTFullExp/'
for city in cities:
    if '.csv' in city:
        df = pd.read_csv(citiesDir+city)
        dataUV = unconstrained_exp(df,separate_income=separate_income)
        dataUV.to_csv(outputDir+city,index=False)


mkdir: cannot create directory ‘unconstrainCTFullExp/’: File exists
mkdir: cannot create directory ‘unconstrainCTFullExpIncome/’: File exists
